# Solve a Task

In this experiment, we are going to construct a modular robot to solve a simple pick and place task. For that purpose, we provide a TrajectoryPlanner class - it takes over iteratively solving the inverse kinematics necessary to reach one or multiple goal poses and generates a trajectory between them.

In [ ]:
import datetime
from pathlib import Path
import time

import numpy as np

from timor.Geometry import Box, ComposedGeometry, Cylinder
from timor.Module import ModuleAssembly, ModulesDB
from timor.Robot import PinRobot
from timor.scenario import CostFunctions, Solution, Tolerance
from timor.scenario.Goals import Reach
from timor.scenario.Obstacle import Obstacle
from timor.scenario.Scenario import Scenario, ScenarioHeader
from timor.utilities.dtypes import Trajectory
from timor.utilities.file_locations import robots as robot_data
from timor.utilities.file_locations import get_module_db_files
import timor.utilities.logging as logging
from timor.utilities.transformation import Transformation
from timor.utilities.tolerated_pose import ToleratedPose
from timor.utilities.spatial import rotX, rotZ

from TrajectoryPlanner import TrajectoryPlanner  # Import the trajectory planner that is provided with this experiment

## Define a Problem
Our robot will be mounted on a desk and should pick an object from that desk an place it elsewhere.

Note that timor is not a path planning library! It's purpose is to generate modular robot models, which can then be integrated simulation suites and various optimizers - so we keep the trajectory planning to the bare minimum here and only define goal poses for this task. The picking and placing itself will not be simulated,  as it should also not affect the efficiency of the robot performing the task.

In [ ]:
# Create a scenario
header = ScenarioHeader(
    scenarioID='Pick & Place',
    tags=['ICRA2023', 'demo'],
    date=datetime.datetime(2022, 9, 15),
    author='Jonathan Külz',
    authorEMail='jonathan.kuelz@tum.de',
    affiliation='Technical University of Munich'
)

# Create a table and a plateau, the obstacles to deal with
height = .4
plateau_height = .04
r_leg = 0.05
dx = 1.6
dy = .8
dz = .05

plateau_placement = Transformation.from_translation([.3 * dx, .2 * dy, height + dz/2 + plateau_height / 2])

surface = Box(dict(x=dx, y=dy, z=dz), pose=Transformation.from_translation([0, 0, height]))
legs = [
    Cylinder(dict(r=r_leg, z=height),
             pose=Transformation.from_translation([i * (dx / 2 - r_leg), j * (dy / 2 - r_leg), height / 2]))
    for i, j in ((-1, -1), (-1, 1), (1, -1), (1, 1))
]
table = Obstacle(ID='table', collision=ComposedGeometry([surface] + legs))
plateau = Obstacle(ID='plateau',
                   collision=Box(dict(x=.3, y=.2, z=plateau_height), pose=plateau_placement))

# Define the goals slightly above the table and the plateau - here would be the grapsing points
first_goal = Reach(goal_id='Reach One',
                   goal_pose=ToleratedPose(nominal=Transformation.from_translation([.55, .2, height + dz + plateau_height]) @ rotX(np.pi),
                                           tolerance=Tolerance.CartesianSpheric.default()))
second_goal = Reach(goal_id='Reach Two',
                    goal_pose=ToleratedPose(nominal=Transformation.from_translation([-.55, .2, height + dz]) @ rotX(np.pi),
                                            tolerance=Tolerance.CartesianSpheric.default()))

scenario = Scenario(header, obstacles=[plateau, table], goals=[first_goal, second_goal])

energy_cost = CostFunctions.MechanicalEnergy()

In [ ]:
scenario.visualize()

Image

## Plan a trajectory
Using the provided TrajectoryPlanner, we are now looking for a trajectory between our two goals:

In [ ]:
base_placement = Transformation.from_translation([0, 0, 1e-3 + height + dz / 2]) @ rotZ(np.pi / 2)

modules = ('base', 'J2', 'J1', 'l_45', 'l_45', 'eef')
db = ModulesDB.from_file(*get_module_db_files('geometric_primitive_modules'))
assembly = ModuleAssembly.from_serial_modules(db, modules)
modular = assembly.to_pin_robot(base_placement=base_placement)
trajectory = TrajectoryPlanner(scenario, modular, trajectory_resolution=.04).solve()
modular_solution = Solution.SolutionTrajectory(trajectory, {'scenarioID': scenario.id}, scenario=scenario, robot=modular, cost_function=energy_cost, base_pose=base_placement)
modular_solution.visualize()

In [ ]:
trajectory.t.shape